<a href="https://colab.research.google.com/github/RashminiA6/UMBC-DATA606-Capstone/blob/main/plant_disease_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from PIL import Image
from zipfile import ZipFile
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import seaborn as sns
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import pandas as pd


In [ ]:
# Unzip the dataset
with ZipFile("/content/drive/MyDrive/plantvillage.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
# Set the path to dataset directory
path_segmented = "plantvillage dataset/segmented"
path_color = "plantvillage dataset/color"
path_grayscale = "plantvillage dataset/grayscale"


In [ ]:
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% for validation
)

In [ ]:
train_generator = datagen.flow_from_directory(
    path_color,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 43456 images belonging to 38 classes.


In [ ]:
val_generator = datagen.flow_from_directory(
    path_color,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 10849 images belonging to 38 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

In [ ]:
# Load the ResNet50 model with pre-trained weights, excluding the top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Add global average pooling
x = Dense(128, activation='relu')(x)  # Fully connected layer
x = Dropout(0.5)(x)  # Dropout for regularization
predictions = Dense(38, activation='softmax')(x)  # Final output layer with 38 categories


In [ ]:
# Build the model
model_resnet = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model
model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Summary of the model
model_resnet.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 128, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 134, 134, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 64, 64, 64)     │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 64, 64, 64)     │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 64, 64, 64)     │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 66, 66, 64)     │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 32, 32, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 32, 32, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 32, 32, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 32, 32, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 32, 32, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 32, 32, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 32, 32, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 32, 32, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 32, 32, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 23,854,886 (91.00 MB)

 Trainable params: 267,174 (1.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# Train the model
history_resnet = model_resnet.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator
)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1358/1358 ━━━━━━━━━━━━━━━━━━━━ 277s 193ms/step - accuracy: 0.1077 - loss: 3.3903 - val_accuracy: 0.1357 - val_loss: 3.1613
Epoch 2/30
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 256s 188ms/step - accuracy: 0.1526 - loss: 3.1766 - val_accuracy: 0.1592 - val_loss: 3.0623
Epoch 3/30
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 258s 189ms/step - accuracy: 0.1708 - loss: 3.1047 - val_accuracy: 0.2100 - val_loss: 2.9971
Epoch 4/30
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 257s 189ms/step - accuracy: 0.1838 - loss: 3.0483 - val_accuracy: 0.2195 - val_loss: 2.9398
Epoch 5/30
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 257s 188ms/step - accuracy: 0.2015 - loss: 2.9939 - val_accuracy: 0.2017 - val_loss: 2.8984
Epoch 6/30
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 257s 188ms/step - accuracy: 0.2110 - loss: 2.9523 - val_accuracy: 0.2341 - val_loss: 2.8431
Epoch 7/30
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 255s 187ms/step - accuracy: 0.2158 - loss: 2.9210 - val_accuracy: 0.2453 - val_loss: 2.8025
Epoch 8/30
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 257s 188ms/step - accuracy: 0.2

In [ ]:
last_epoch = len(history_resnet.history['loss'])

history_resnet = model_resnet.fit(
    train_generator,
    epochs= 40,
    initial_epoch=last_epoch,
    validation_data=val_generator
)

Epoch 31/40
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 259s 190ms/step - accuracy: 0.2675 - loss: 2.6520 - val_accuracy: 0.3176 - val_loss: 2.4554
Epoch 32/40
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 258s 189ms/step - accuracy: 0.2652 - loss: 2.6518 - val_accuracy: 0.3056 - val_loss: 2.4595
Epoch 33/40
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 261s 191ms/step - accuracy: 0.2648 - loss: 2.6622 - val_accuracy: 0.3104 - val_loss: 2.4383
Epoch 34/40
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 261s 191ms/step - accuracy: 0.2700 - loss: 2.6420 - val_accuracy: 0.3078 - val_loss: 2.4553
Epoch 35/40
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 259s 190ms/step - accuracy: 0.2661 - loss: 2.6508 - val_accuracy: 0.3182 - val_loss: 2.4309
Epoch 36/40
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 258s 189ms/step - accuracy: 0.2671 - loss: 2.6361 - val_accuracy: 0.3126 - val_loss: 2.4393
Epoch 37/40
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 258s 189ms/step - accuracy: 0.2738 - loss: 2.6252 - val_accuracy: 0.3078 - val_loss: 2.4476
Epoch 38/40
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 254s 186ms/s